In [52]:
import pandas as pd
import xgboost as xgb
import category_encoders as ce
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


In [2]:
sample_submission = pd.read_csv(r"data/sample_submission.csv")
# 提出用ファイルの中身の確認
sample_submission

,ID,取引価格（総額）_log
0,1000000,0
1,1000001,0
2,1000062,0
3,1000114,0
4,1000116,0
...,...,...
23146,47006024,0
23147,47006082,0
23148,47006216,0
23149,47006238,0


In [3]:
#input file name
input_file_name = 'data/data_explanation.xlsx'
#xls book Open (xls, xlsxのどちらでも可能)
input_book = pd.ExcelFile(input_file_name)
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = input_book.sheet_names
#lenでシートの総数を確認
num_sheet = len(input_sheet_name)
#シートの数とシートの名前のリストの表示
print ("Sheet の数:", num_sheet)
print (input_sheet_name)

Sheet の数: 2
['ファイル説明', 'カラム説明']


In [4]:
#DataFrameとしてsheet1枚のデータ(ファイル説明)を読込み
input_sheet_df = input_book.parse(input_sheet_name[0])
#読み込んだシート名の確認
print("Sheet name:", input_sheet_name[0])
#読み込んだシートの先頭10行を表示
input_sheet_df[['ファイル名', '内容']].head()

Sheet name: ファイル説明


,ファイル名,内容
0,train.zip\n └─*.csv,2020年第４四半期より前に取引された中古マンションの価格データ
1,test.csv,2020年第４四半期、2021年第１四半期に取引された中古マンションの価格データ
2,sample_submission.csv,投稿データフォーマット


In [5]:
#DataFrameとしてsheet1枚のデータ(ファイル説明)を読込み
input_sheet_df = input_book.parse(input_sheet_name[1])
#読み込んだシート名の確認
print("Sheet name:", input_sheet_name[1])

Sheet name: カラム説明


In [6]:
#読み込んだシートの先頭10行を表示
columns_list = input_sheet_df.columns
input_sheet_df[['{}'.format(columns_list[0]), '{}'.format(columns_list[1])]]

,カラム名,内容
0,ID,取引ID
1,種類,不動産の種類
2,地域,周辺地域の状況 ※本データではnull
3,市区町村コード,市区町村コード
4,都道府県名,都道府県名
5,市区町村名,市区町村名
6,地区名,地区名
7,最寄駅：名称,最寄りの鉄道駅の名称
8,最寄駅：距離（分）,当該地から最寄りの鉄道駅（地下駅の場合には地表への出入口）までの時間距離（分）
9,間取り,専有部分の間取りを表示しています。なお、間取りが把握できないものは、空欄になっています。\n...


In [7]:
test_df = pd.read_csv('data/test.csv')
print(test_df.head())
print('columns:{}'.format(test_df.columns))

        ID        種類  地域  市区町村コード 都道府県名   市区町村名  地区名 最寄駅：名称 最寄駅：距離（分）   間取り  \
0  1000000  中古マンション等 NaN     1101   北海道  札幌市中央区  旭ケ丘   円山公園        25  ４ＬＤＫ   
1  1000001  中古マンション等 NaN     1101   北海道  札幌市中央区  旭ケ丘   円山公園        20  ４ＬＤＫ   
2  1000062  中古マンション等 NaN     1101   北海道  札幌市中央区  大通西  西１１丁目         3  ２ＬＤＫ   
3  1000114  中古マンション等 NaN     1101   北海道  札幌市中央区  大通西  西１８丁目         2  １ＬＤＫ   
4  1000116  中古マンション等 NaN     1101   北海道  札幌市中央区  大通西  西１８丁目         4  ２ＬＤＫ   

   ... 今後の利用目的  前面道路：方位  前面道路：種類  前面道路：幅員（ｍ）         都市計画 建ぺい率（％） 容積率（％）  \
0  ...      住宅      NaN      NaN         NaN  第１種低層住居専用地域    40.0   80.0   
1  ...      住宅      NaN      NaN         NaN       近隣商業地域    80.0  200.0   
2  ...      住宅      NaN      NaN         NaN         商業地域    80.0  600.0   
3  ...      住宅      NaN      NaN         NaN         商業地域    80.0  400.0   
4  ...      住宅      NaN      NaN         NaN       近隣商業地域    80.0  300.0   

         取引時点   改装  取引の事情等  
0  2021年第１四半期  未改装     NaN  
1  2021年第１

In [8]:
import glob
train_files = glob.glob('data/train/*.csv')
print(train_files)

['data/train/40.csv', 'data/train/41.csv', 'data/train/43.csv', 'data/train/42.csv', 'data/train/46.csv', 'data/train/47.csv', 'data/train/45.csv', 'data/train/44.csv', 'data/train/37.csv', 'data/train/23.csv', 'data/train/22.csv', 'data/train/36.csv', 'data/train/08.csv', 'data/train/20.csv', 'data/train/34.csv', 'data/train/35.csv', 'data/train/21.csv', 'data/train/09.csv', 'data/train/25.csv', 'data/train/31.csv', 'data/train/19.csv', 'data/train/18.csv', 'data/train/30.csv', 'data/train/24.csv', 'data/train/32.csv', 'data/train/26.csv', 'data/train/27.csv', 'data/train/33.csv', 'data/train/16.csv', 'data/train/02.csv', 'data/train/03.csv', 'data/train/17.csv', 'data/train/29.csv', 'data/train/01.csv', 'data/train/15.csv', 'data/train/14.csv', 'data/train/28.csv', 'data/train/04.csv', 'data/train/10.csv', 'data/train/38.csv', 'data/train/39.csv', 'data/train/11.csv', 'data/train/05.csv', 'data/train/13.csv', 'data/train/07.csv', 'data/train/06.csv', 'data/train/12.csv']


In [9]:
train_df_list = []
for file in train_files:
    df = pd.read_csv(file)
    train_df_list.append(df)
train_df = pd.concat(train_df_list)

/Users/masato/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
print('train_df:\n{}'.format(train_df.columns))
print('test_df:\n{}'.format(test_df.columns))
print('sample_df:\n{}'.format(sample_submission.columns))

train_df:
Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')
test_df:
Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等'],
      dtype='object')
sample_df:
Index(['ID', '取引価格（総額）_log'], dtype='object')


In [11]:
train_df['test'] = 0
test_df['test'] = 1
df = pd.concat([train_df, test_df])

In [12]:

def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[-1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[-1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[-1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    cols = ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
    ce_df = ce.OrdinalEncoder(cols=cols, handle_unknown='impute')
    df = ce_df.fit_transform(df)
    
    return df
    
df = data_pre(df)
train_df = df[df['test'] == 0]
test_df = df[df['test'] == 1]


In [13]:
train_df = train_df.drop("test", axis = 1)
test_df = test_df.drop(["test", "取引価格（総額）_log"], axis = 1)
train_df.head()

,ID,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,40012297,40131,1,1,1,6.0,1,60.0,25.0,1,1,1,1,NaN,NaN,2017.25,1,1,6.919078
1,40067189,40133,1,2,2,4.0,2,25.0,8.0,1,1,2,2,80.0,400.0,2014.75,1,1,7.230449
2,40019754,40133,1,3,3,3.0,3,65.0,5.0,1,2,1,2,80.0,400.0,2019.75,1,1,7.643453
3,40097300,40132,1,4,4,16.0,4,15.0,26.0,1,1,2,3,80.0,300.0,2013.75,1,1,6.544068
4,40017646,40133,1,5,5,8.0,2,60.0,17.0,1,1,1,2,80.0,400.0,2016.25,1,1,7.462398


In [14]:
def return_predict_by_XGBoost(train_df, test_df, sample_submission):
    df_train, df_val =train_test_split(train_df, test_size=0.2)

    col = "取引価格（総額）_log"
    train_y = df_train[col]
    train_x = df_train.drop(col, axis=1)

    val_y = df_val[col]
    val_x = df_val.drop(col, axis=1)

    train_data = xgb.DMatrix(train_x, label=train_y)
    eval_data = xgb.DMatrix(val_x, label=val_y)

    xgb_params = {
        "objective": "reg:squarederror",
        'eval_metric': "mae"
        }
    evals = [(train_data, 'train'), (eval_data, 'eval')]

    gbm = xgb.train(
        xgb_params,
        train_data,
        num_boost_round=100,
        early_stopping_rounds=10,
        evals=evals,
        )
    
    predict_X = xgb.DMatrix(test_df)
    y_pred = gbm.predict(predict_X)
    columns = sample_submission.columns
    for i in range(len(y_pred)):
        sample_submission[columns[1]][i] = y_pred[i]
        
    return sample_submission

In [15]:
sample_submission = return_predict_by_XGBoost(train_df, test_df, sample_submission)
sample_submission.to_csv('submission2.csv', index=False)

[0]	train-mae:4.70591	eval-mae:4.70574
[1]	train-mae:3.29432	eval-mae:3.29427
[2]	train-mae:2.30622	eval-mae:2.30630
[3]	train-mae:1.61459	eval-mae:1.61482
[4]	train-mae:1.13086	eval-mae:1.13110
[5]	train-mae:0.79396	eval-mae:0.79419
[6]	train-mae:0.55998	eval-mae:0.56012
[7]	train-mae:0.39878	eval-mae:0.39882
[8]	train-mae:0.28993	eval-mae:0.29007
[9]	train-mae:0.21923	eval-mae:0.21949
[10]	train-mae:0.17477	eval-mae:0.17513
[11]	train-mae:0.14847	eval-mae:0.14897
[12]	train-mae:0.13321	eval-mae:0.13374
[13]	train-mae:0.12451	eval-mae:0.12508
[14]	train-mae:0.11971	eval-mae:0.12026
[15]	train-mae:0.11563	eval-mae:0.11617
[16]	train-mae:0.11316	eval-mae:0.11372
[17]	train-mae:0.11167	eval-mae:0.11227
[18]	train-mae:0.11046	eval-mae:0.11105
[19]	train-mae:0.10974	eval-mae:0.11035
[20]	train-mae:0.10899	eval-mae:0.10961
[21]	train-mae:0.10852	eval-mae:0.10916
[22]	train-mae:0.10790	eval-mae:0.10856
[23]	train-mae:0.10726	eval-mae:0.10797
[24]	train-mae:0.10636	eval-mae:0.10708
[25]	train

In [42]:
### すでに学習データは分かれているためこの操作はいらない
# # 学習データとテストデータに分ける　->すべてがひとかたまりの場合この操作をする
# X_train, X_test, y_train, y_test = train_test_split(df_X, y,
#                                                     test_size=0.2,
#                                                     random_state=0,
#                                                     stratify=y)
test_df = pd.read_csv('data/test.csv')
train_files = glob.glob('data/train/*.csv')
train_df_list = []
for file in train_files:
    df = pd.read_csv(file)
    train_df_list.append(df)
train_df = pd.concat(train_df_list)
train_df['test'] = 0
test_df['test'] = 1
df = pd.concat([train_df, test_df])
df = data_pre(df)
train_df = df[df['test'] == 0]
test_df = df[df['test'] == 1]
train_df = train_df.drop("test", axis = 1)
test_df = test_df.drop(["test", "取引価格（総額）_log"], axis = 1)
col = "取引価格（総額）_log"
df_X = train_df.drop(col, axis = 1)
y = train_df[col]
# 学習データとテストデータに分ける　->すべてがひとかたまりの場合この操作をする
X_train, X_test, y_train, y_test = train_test_split(df_X, y,
                                                    test_size=0.2,
                                                    random_state=0)
# 学習データを、学習用と検証用に分ける
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train,
                                                    test_size=0.2,
                                                    random_state=2)

# データを格納する
# テスト用
xgb_test = xgb.DMatrix(X_test, label=y_test)
# 5-fold CVモデルの学習
# 5つのモデルを保存するリストの初期化
models = []
#accuracyを保存するNumPy配列の初期化
accuracies = np.array([])

# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って5分割する）
K_fold = KFold(n_splits=5, shuffle=True,  random_state=42)

# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
    
    # 学習用
    xgb_train = xgb.DMatrix(X_train_cv, label=y_train_cv)

    # 検証用
    xgb_eval = xgb.DMatrix(X_eval_cv, label=y_eval_cv)
   
    
    # パラメータを設定
    xgb_params = {
        "objective": "reg:squarederror",
        'eval_metric': "mae"
        }
    
    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst = xgb.train(xgb_params,                        # 上記で設定したパラメーター
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=300,               # 学習の回数
                    early_stopping_rounds=10,          # アーリーストッピング
                    evals=evals,                       # 上記で設定した検証用データ
                    evals_result=evaluation_results,   # 上記で設定した箱
                    verbose_eval=1                     # 学習の経過の表示(非表示)
                    ) 
    
    # テストデータで予測する
    y_pred = bst.predict(xgb_test, ntree_limit=bst.best_ntree_limit)
#     y_pred_max = np.argmax(y_pred, axis=1) #マルチクラス用のAccuracy
    

    # Accuracy を計算する
#     accuracy = sum(y_test == y_pred_max) / len(y_test) #マルチクラス用
    accuracy = np.sqrt(mean_squared_error(y_test, y_pred))
    #print('accuracy:', accuracy)
    
#     # Logloss を計算する
#     y_pred_ans = []
#     # 正解ラベルの確率
#     for i in range(len(y_pred)):
#         y_pred_ans.append(y_pred[i, y_test[i]])

#     logloss = -np.sum(np.log(y_pred_ans)) / len(y_pred_ans)
    print('accuracy:', accuracy)
    
    # 学習が終わったモデルをリストに入れておく
    models.append(bst) 
    # 学習結果をNumPy配列に入れておく
    accuracies = np.append(accuracies, accuracy)
    
# 正解ラベルの確率とLogloss（平均値）
print('accuracy_ave: ', np.mean(accuracies))

/Users/masato/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


[0]	train-mae:4.70625	eval-mae:4.70462
[1]	train-mae:3.29465	eval-mae:3.29359
[2]	train-mae:2.30648	eval-mae:2.30579
[3]	train-mae:1.61481	eval-mae:1.61428
[4]	train-mae:1.13099	eval-mae:1.13078
[5]	train-mae:0.79405	eval-mae:0.79412
[6]	train-mae:0.56010	eval-mae:0.56012
[7]	train-mae:0.39876	eval-mae:0.39858
[8]	train-mae:0.28993	eval-mae:0.28966
[9]	train-mae:0.21915	eval-mae:0.21899
[10]	train-mae:0.17492	eval-mae:0.17479
[11]	train-mae:0.14869	eval-mae:0.14850
[12]	train-mae:0.13374	eval-mae:0.13354
[13]	train-mae:0.12372	eval-mae:0.12359
[14]	train-mae:0.11841	eval-mae:0.11836
[15]	train-mae:0.11510	eval-mae:0.11514
[16]	train-mae:0.11298	eval-mae:0.11305
[17]	train-mae:0.11148	eval-mae:0.11164
[18]	train-mae:0.11053	eval-mae:0.11070
[19]	train-mae:0.10942	eval-mae:0.10959
[20]	train-mae:0.10866	eval-mae:0.10887
[21]	train-mae:0.10757	eval-mae:0.10784
[22]	train-mae:0.10702	eval-mae:0.10732
[23]	train-mae:0.10612	eval-mae:0.10644
[24]	train-mae:0.10569	eval-mae:0.10606
[25]	train

[203]	train-mae:0.08225	eval-mae:0.08622
[204]	train-mae:0.08218	eval-mae:0.08617
[205]	train-mae:0.08215	eval-mae:0.08614
[206]	train-mae:0.08211	eval-mae:0.08611
[207]	train-mae:0.08206	eval-mae:0.08607
[208]	train-mae:0.08201	eval-mae:0.08604
[209]	train-mae:0.08198	eval-mae:0.08603
[210]	train-mae:0.08195	eval-mae:0.08601
[211]	train-mae:0.08193	eval-mae:0.08601
[212]	train-mae:0.08190	eval-mae:0.08599
[213]	train-mae:0.08184	eval-mae:0.08596
[214]	train-mae:0.08179	eval-mae:0.08592
[215]	train-mae:0.08176	eval-mae:0.08591
[216]	train-mae:0.08172	eval-mae:0.08589
[217]	train-mae:0.08170	eval-mae:0.08588
[218]	train-mae:0.08169	eval-mae:0.08588
[219]	train-mae:0.08161	eval-mae:0.08583
[220]	train-mae:0.08157	eval-mae:0.08580
[221]	train-mae:0.08153	eval-mae:0.08578
[222]	train-mae:0.08145	eval-mae:0.08573
[223]	train-mae:0.08142	eval-mae:0.08571
[224]	train-mae:0.08134	eval-mae:0.08566
[225]	train-mae:0.08128	eval-mae:0.08561
[226]	train-mae:0.08122	eval-mae:0.08558
[227]	train-mae:

/Users/masato/miniforge3/envs/ds_env/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[0]	train-mae:4.70609	eval-mae:4.70556
[1]	train-mae:3.29452	eval-mae:3.29438
[2]	train-mae:2.30642	eval-mae:2.30645
[3]	train-mae:1.61480	eval-mae:1.61494
[4]	train-mae:1.13105	eval-mae:1.13122
[5]	train-mae:0.79415	eval-mae:0.79440
[6]	train-mae:0.56015	eval-mae:0.56032
[7]	train-mae:0.39894	eval-mae:0.39908
[8]	train-mae:0.29008	eval-mae:0.29016
[9]	train-mae:0.21899	eval-mae:0.21918
[10]	train-mae:0.17506	eval-mae:0.17550
[11]	train-mae:0.14871	eval-mae:0.14925
[12]	train-mae:0.13400	eval-mae:0.13460
[13]	train-mae:0.12532	eval-mae:0.12599
[14]	train-mae:0.11986	eval-mae:0.12059
[15]	train-mae:0.11613	eval-mae:0.11689
[16]	train-mae:0.11389	eval-mae:0.11461
[17]	train-mae:0.11247	eval-mae:0.11323
[18]	train-mae:0.11158	eval-mae:0.11236
[19]	train-mae:0.11059	eval-mae:0.11140
[20]	train-mae:0.10880	eval-mae:0.10962
[21]	train-mae:0.10736	eval-mae:0.10823
[22]	train-mae:0.10655	eval-mae:0.10739
[23]	train-mae:0.10621	eval-mae:0.10705
[24]	train-mae:0.10551	eval-mae:0.10638
[25]	train

...

[202]	train-mae:0.08182	eval-mae:0.08647
[203]	train-mae:0.08179	eval-mae:0.08646
[204]	train-mae:0.08177	eval-mae:0.08645
[205]	train-mae:0.08173	eval-mae:0.08642
[206]	train-mae:0.08167	eval-mae:0.08638
[207]	train-mae:0.08162	eval-mae:0.08636
[208]	train-mae:0.08156	eval-mae:0.08631
[209]	train-mae:0.08147	eval-mae:0.08623
[210]	train-mae:0.08141	eval-mae:0.08618
[211]	train-mae:0.08134	eval-mae:0.08613
[212]	train-mae:0.08130	eval-mae:0.08611
[213]	train-mae:0.08127	eval-mae:0.08611
[214]	train-mae:0.08124	eval-mae:0.08609
[215]	train-mae:0.08123	eval-mae:0.08609
[216]	train-mae:0.08120	eval-mae:0.08606
[217]	train-mae:0.08117	eval-mae:0.08604
[218]	train-mae:0.08113	eval-mae:0.08603
[219]	train-mae:0.08110	eval-mae:0.08600
[220]	train-mae:0.08106	eval-mae:0.08598
[221]	train-mae:0.08101	eval-mae:0.08594
[222]	train-mae:0.08095	eval-mae:0.08589
[223]	train-mae:0.08092	eval-mae:0.08589
[224]	train-mae:0.08088	eval-mae:0.08586
[225]	train-mae:0.08084	eval-mae:0.08584
[226]	train-mae:

/Users/masato/miniforge3/envs/ds_env/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


accuracy: 0.13556092575234568
[0]	train-mae:4.70580	eval-mae:4.70687
[1]	train-mae:3.29431	eval-mae:3.29514
[2]	train-mae:2.30624	eval-mae:2.30686
[3]	train-mae:1.61465	eval-mae:1.61506
[4]	train-mae:1.13094	eval-mae:1.13127
[5]	train-mae:0.79404	eval-mae:0.79439
[6]	train-mae:0.56000	eval-mae:0.56055
[7]	train-mae:0.39862	eval-mae:0.39962
[8]	train-mae:0.28979	eval-mae:0.29115
[9]	train-mae:0.21904	eval-mae:0.22063
[10]	train-mae:0.17511	eval-mae:0.17684
[11]	train-mae:0.14903	eval-mae:0.15087
[12]	train-mae:0.13410	eval-mae:0.13598
[13]	train-mae:0.12526	eval-mae:0.12714
[14]	train-mae:0.11972	eval-mae:0.12168
[15]	train-mae:0.11510	eval-mae:0.11707
[16]	train-mae:0.11295	eval-mae:0.11492
[17]	train-mae:0.11159	eval-mae:0.11357
[18]	train-mae:0.11056	eval-mae:0.11257
[19]	train-mae:0.10975	eval-mae:0.11183
[20]	train-mae:0.10869	eval-mae:0.11084
[21]	train-mae:0.10796	eval-mae:0.11013
[22]	train-mae:0.10708	eval-mae:0.10925
[23]	train-mae:0.10668	eval-mae:0.10888
[24]	train-mae:0.106

[202]	train-mae:0.08186	eval-mae:0.08733
[203]	train-mae:0.08183	eval-mae:0.08732
[204]	train-mae:0.08177	eval-mae:0.08729
[205]	train-mae:0.08172	eval-mae:0.08725
[206]	train-mae:0.08168	eval-mae:0.08723
[207]	train-mae:0.08164	eval-mae:0.08722
[208]	train-mae:0.08155	eval-mae:0.08713
[209]	train-mae:0.08145	eval-mae:0.08705
[210]	train-mae:0.08138	eval-mae:0.08700
[211]	train-mae:0.08132	eval-mae:0.08696
[212]	train-mae:0.08124	eval-mae:0.08689
[213]	train-mae:0.08119	eval-mae:0.08685
[214]	train-mae:0.08116	eval-mae:0.08683
[215]	train-mae:0.08113	eval-mae:0.08681
[216]	train-mae:0.08108	eval-mae:0.08677
[217]	train-mae:0.08105	eval-mae:0.08676
[218]	train-mae:0.08101	eval-mae:0.08673
[219]	train-mae:0.08095	eval-mae:0.08669
[220]	train-mae:0.08092	eval-mae:0.08669
[221]	train-mae:0.08088	eval-mae:0.08667
[222]	train-mae:0.08084	eval-mae:0.08666
[223]	train-mae:0.08081	eval-mae:0.08662
[224]	train-mae:0.08080	eval-mae:0.08662
[225]	train-mae:0.08077	eval-mae:0.08661
[226]	train-mae:

/Users/masato/miniforge3/envs/ds_env/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [41]:
models

In [55]:
# スタッキングによる予測
# X_eval = xgb.DMatrix(X_eval)
# X_test = xgb.DMatrix(X_test)

# 第1段階の予測値(この後、メタモデルの入力に使用)
first_pred_1 = models[0].predict(X_eval)
first_pred_2 = models[1].predict(X_eval)
first_pred_3 = models[2].predict(X_eval)
first_pred_4 = models[3].predict(X_eval)
first_pred_5 = models[4].predict(X_eval)

#第1段階の予測値を積み重ねる
stack_pred = np.column_stack((first_pred_1, first_pred_2, first_pred_3, first_pred_4, first_pred_5))

# メタモデルの学習 
meta_model = LinearRegression()
meta_model.fit(stack_pred, y_eval)

# 結果の検証 
test_pred_1 = models[0].predict(X_test)
test_pred_2 = models[1].predict(X_test)
test_pred_3 = models[2].predict(X_test)
test_pred_4 = models[3].predict(X_test)
test_pred_5 = models[4].predict(X_test)


# 各モデルの検証データを積み重ねる
stack_test_pred = np.column_stack((test_pred_1, test_pred_2, test_pred_3, test_pred_4, test_pred_5))

# スタッキングの検証
meta_test_pred = meta_model.predict(stack_test_pred)
print ("メタモデルの平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, meta_test_pred)))

メタモデルの平均2乗誤差: 0.0174


In [56]:
print(stack_test_pred)

[[7.0072575 7.013508  7.024407  7.03462   7.028847 ]
 [6.850501  6.8218203 6.8254232 6.8353686 6.831946 ]
 [7.146479  7.1623316 7.170465  7.13252   7.199555 ]
 ...
 [6.9842234 6.921456  6.8991375 7.0063977 6.915595 ]
 [7.24081   7.3236847 7.3066106 7.3150125 7.3098135]
 [7.212639  7.2683473 7.256615  7.23775   7.234121 ]]


In [57]:
X_TEST = xgb.DMatrix(test_df)
# 結果の検証 
test_pred_1 = models[0].predict(X_TEST)
test_pred_2 = models[1].predict(X_TEST)
test_pred_3 = models[2].predict(X_TEST)
test_pred_4 = models[3].predict(X_TEST)
test_pred_5 = models[4].predict(X_TEST)
# 各モデルの検証データを積み重ねる
stack_test_pred = np.column_stack((test_pred_1, test_pred_2, test_pred_3, test_pred_4, test_pred_5))
# スタッキングの検証
meta_test_pred = meta_model.predict(stack_test_pred)

In [58]:
meta_test_pred

array([7.042633 , 7.3774915, 7.448989 , ..., 7.515641 , 7.389268 ,
       7.39146  ], dtype=float32)

In [59]:
columns = sample_submission.columns
for i in range(len(meta_test_pred)):
    sample_submission[columns[1]][i] = meta_test_pred[i]
sample_submission.to_csv('submission3.csv', index=False)